In [1]:
!pip install transformers datasets evaluate accelerate numpy sentencepiece --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [2]:
from datasets import load_dataset
import evaluate
import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Load dataset
dataset = load_dataset("parquet", data_files={
    "train": "/kaggle/input/translation/data/train-00000-of-00001.parquet",
    "validation": "/kaggle/input/translation/data/validation-00000-of-00001.parquet", 
    "test": "/kaggle/input/translation/data/test-00000-of-00001.parquet"
})

# Subset the data
train_size, val_size, test_size = 50000, 5000, 2000
dataset["train"] = dataset["train"].select(range(train_size))
dataset["validation"] = dataset["validation"].select(range(val_size))
dataset["test"] = dataset["test"].select(range(test_size))


2025-06-22 11:19:27.783806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750591167.966335      91 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750591168.017250      91 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
# Filter empty or very long samples
def is_valid_pair(example):
    source, target = str(example["source"]).strip(), str(example["target"]).strip()
    return 0 < len(source) < 300 and 0 < len(target) < 300

dataset = dataset.filter(is_valid_pair)

print("Dataset sizes after filtering:")
print(f"Train: {len(dataset['train'])}")
print(f"Validation: {len(dataset['validation'])}")
print(f"Test: {len(dataset['test'])}")

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset sizes after filtering:
Train: 49746
Validation: 4977
Test: 1991


In [4]:
# Load tokenizer and model
checkpoint_path = "/kaggle/working/namaste"
tokenizer = MT5Tokenizer.from_pretrained(checkpoint_path)
model = MT5ForConditionalGeneration.from_pretrained(checkpoint_path)
model.config.use_cache = False
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [ ]:
# Preprocessing function
def preprocess_function(examples):
    inputs = ["translate Nepali to English: " + str(src).strip() for src in examples["source"]]
    targets = [str(tgt).strip() for tgt in examples["target"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=300, truncation=True, padding="max_length")
    model_inputs["labels"] = [
        [(token_id if token_id != tokenizer.pad_token_id else -100) for token_id in label]
        for label in model_inputs["labels"]
    ]
    return model_inputs

In [ ]:
# Tokenize datasets
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
tokenized_dataset.set_format(type="torch")

In [ ]:
bleu = evaluate.load("bleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    bleu_score = bleu.compute(predictions=decoded_preds, references=decoded_labels)["bleu"]
    return {"bleu": bleu_score}


In [ ]:
# Data collator
class CustomDataCollator(DataCollatorForSeq2Seq):
    def __call__(self, features, return_tensors=None):
        batch = super().__call__(features, return_tensors=return_tensors)
        # Convert labels to tensor safely
        if isinstance(batch["labels"], list):
            labels = [np.asarray(x, dtype=np.int64) for x in batch["labels"]]
            batch["labels"] = torch.tensor(labels)
        return {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}



data_collator = CustomDataCollator(tokenizer=tokenizer, model=model)


In [ ]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5-npi-en",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    logging_dir="./logs",
    logging_steps=500,
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    generation_max_length=300,
    weight_decay=0.01,
    save_total_limit=1,
    warmup_steps=500,
    predict_with_generate=True,
    dataloader_pin_memory=False,
    dataloader_num_workers=0,
    remove_unused_columns=False,
    use_cpu=False,
    fp16=True,
    report_to=[],
)


In [ ]:
class CustomTrainer(Seq2SeqTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.train_losses = []
        self.eval_losses = []
        self.bleu_scores = []

    def log(self, logs, step=None, **kwargs):
        super().log(logs)
        if "loss" in logs:
            self.train_losses.append(logs["loss"])

    def evaluate(self, *args, **kwargs):
        metrics = super().evaluate(*args, **kwargs)
        self.eval_losses.append(metrics.get("eval_loss", None))
        return metrics

    def save_metrics_plot(self):
        epochs = list(range(0, len(self.eval_losses)))

        plt.figure(figsize=(10, 6))
        plt.plot(self.train_losses, label="Training Loss", marker="o")
        plt.plot(epochs, self.eval_losses, label="Validation Loss", marker="o")
        plt.xlabel("Epoch")
        plt.ylabel("Metric Value")
        plt.title("Training Metrics")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig("training_metrics.png")
        plt.show()


In [ ]:
# Trainer setup
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
trainer.train(resume_from_checkpoint="/kaggle/working/mt5-npi-en/checkpoint-31095")


In [ ]:
                                                                                                                                                                                                                                                                                                    llo

In [ ]:
trainer.save_model("/kaggle/working/modelbhai")
tokenizer.save_pretrained("/kaggle/working/modelbhai ")
trainer.save_metrics_plot()

In [ ]:
test_results = trainer.evaluate(eval_dataset=tokenized_dataset["test"])
print("Test Evaluation:", test_results)

In [ ]:
# Evaluation with BLEU
bleu = evaluate.load("bleu")
model.eval()
sources, references = dataset["test"]["source"], dataset["test"]["target"]
predictions, batch_size = [], 4

for i in tqdm(range(0, len(sources), batch_size), desc="Generating translations"):
    batch_inputs = ["translate Nepali to English: " + s for s in sources[i:i+batch_size]]
    inputs = tokenizer(batch_inputs, return_tensors="pt", padding=True, truncation=True, max_length=280).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=300, num_beams=4)
    predictions.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))

# Print samples and compute BLEU
for i in range(min(20, len(sources))):
    print(f"\nSource: {sources[i]}\nReference: {references[i]}\nPredicted: {predictions[i]}")
print("Test BLEU:", bleu.compute(predictions=[p.strip() for p in predictions], references=[[r.strip()] for r in references])["bleu"])
